# Using HTML Tables
I was looking at some music MIDI notation today, and realized I wanted to get a table from the internet into a PowerShell table format. So I took this opportunity to figure out how!

I found most of this already written from Lee Holmes's blog. 

<pre># Source: https://www.leeholmes.com/blog/2015/01/05/extracting-tables-from-powershells-invoke-webrequest/

# Filename GetHTMLTable.ps1

param(
    [Parameter(Mandatory = $true)]
    [Microsoft.PowerShell.Commands.HtmlWebResponseObject] $WebRequest,
    [Parameter(Mandatory = $true)]
    [int] $TableNumber
)

## Extract the tables out of the web request
$tables = @($WebRequest.ParsedHtml.getElementsByTagName("TABLE"))
$table = $tables[$TableNumber]
$titles = @()
$rows = @($table.Rows)

## Go through all of the rows in the table
foreach($row in $rows)
{
    $cells = @($row.Cells)

    ## If we've found a table header, remember its titles
    if($cells[0].tagName -eq "TH")
    {
        $titles = @($cells | % { ("" + $_.InnerText).Trim() })
        continue
    }

    ## If we haven't found any table headers, use the first row and hope that work
    if(-not $titles)
    {
        $titles = @($cells | % { ("" + $_.InnerText).Trim() })
    }

    ## Now go through the cells in the the row. For each, try to find the
    ## title that represents that column and create a hashtable mapping those
    ## titles to content

    $resultObject = [Ordered] @{}
    for($counter = 0; $counter -lt $cells.Count; $counter++)
    {
        $title = $titles[$counter]
        if(-not $title) { continue }
        $resultObject[$title] = ("" + $cells[$counter].InnerText).Trim()
    }

    ## And finally cast that hashtable to a PSCustomObject
    [PSCustomObject] $resultObject
}</pre>

In [8]:
# Use of the above:
!powershell $r = Invoke-WebRequest 'https://www.presidentsusa.net/presvplist.html' ;\
$Table = ..\scripts\GetHTMLTable.ps1 $r -TableNumber 0 ;\
$Table ^| Format-Table


President                        Term Began              Term Ended                           
---------                        ----------              ----------                           
1. George Washington             April 30, 1789          March 4, 1797                        
2. John Adams                    March 4, 1797           March 4, 1801                        
3. Thomas Jefferson              March 4, 1801           March 4, 1809                        
4. James Madison                 March 4, 1809           March 4, 1817                        
5. James Monroe                  March 4, 1817           March 4, 1825                        
6. John Quincy Adams             March 4, 1825           March 4, 1829                        
7. Andrew Jackson                March 4, 1829           March 4, 1837                        
8. Martin Van Buren              March 4, 1837           March 4, 1841                        
9. William Henry Harrison        March 4, 1841   